**This notebook is covered by the following [license](License.ipynb)  This note must not be removed**

# Installing a restricted account

# __WARNING__

It is extremely easy to create a 

see [Jeff Geeling: Everyone might be cluster admin your kubernetes cluster](https://www.jeffgeerling.com/blog/2020/everyone-might-be-cluster-admin-your-kubernetes-cluster)


Following [Armory Spinnaker Installation](https://docs.armory.io/spinnaker-install-admin-guides/manual-service-account/)

In [1]:
SOURCENAMESPACE=my-namespace
TARGETNAMESPACE=$SOURCENAMESPACE
SERVICEACCOUNT=my-serviceaccount
ROLE=my-role
ROLEBINDING=my-rolebinding

In [2]:
kubectl create namespace $TARGETNAMESPACE

namespace/my-namespace created


In [3]:
kubectl create serviceaccount --namespace=$SOURCENAMESPACE $SERVICEACCOUNT

serviceaccount/my-serviceaccount created


In [4]:
kubectl get  serviceaccount --namespace=$SOURCENAMESPACE

NAME                SECRETS   AGE
default             1         8s
my-serviceaccount   1         3s


## Create ClusterAdmin Binding

Creating a **ClusterRoleBinding** to the **ClusterRole** named **cluster-admin** is dangerous. 

Effectively the serviceaccount becomes allmighty

**It it essential to have different accounts, otherwise the cannot be removed later**



**don't**
```yaml
kubectl delete -f - <<EOF
apiVersion: rbac.authorization.k8s.io/v1beta1
kind: ClusterRoleBinding
metadata:
  name: my-admin
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: ClusterRole
  name: cluster-admin
subjects:
- kind: ServiceAccount
  name: $SERVICEACCOUNT
  namespace: $SOURCENAMESPACE
```

## Namespace restricted roles

Try to use only roles with **RoleBindings** to a certain namespace. The **Role** and their **rules** can be changed later.

[Kubernetes RBAC](https://kubernetes.io/docs/reference/access-authn-authz/rbac/)

In [5]:
kubectl create -f - <<EOF
---
apiVersion: rbac.authorization.k8s.io/v1
kind: Role
metadata:
  name: ${ROLE}
  namespace: ${TARGETNAMESPACE} # Should be namespace you are granting access to
rules:
- apiGroups: ["*"]
  resources: ["*"]
  verbs: ["*"]
---
apiVersion: rbac.authorization.k8s.io/v1
kind: RoleBinding
metadata:
  name: ${ROLEBINDING}
  namespace: $TARGETNAMESPACE # Should be namespace you are granting access to
roleRef:
  apiGroup: rbac.authorization.k8s.io
  kind: Role
  name: ${ROLE} # Should match name of Role
subjects:
- namespace: ${SOURCENAMESPACE} # Should match namespace where SA lives
  kind: ServiceAccount
  name: ${SERVICEACCOUNT} # Should match service account name, above
EOF

role.rbac.authorization.k8s.io/my-role created
rolebinding.rbac.authorization.k8s.io/my-rolebinding created


Please edit to your needs


In [6]:
# fix kube config
sed -i 's/\"\"/\"minikube\"/' /root/.kube/config
cat /root/.kube/config

apiVersion: v1
clusters:
- cluster:
    certificate-authority: /var/lib/minikube/certs/ca.crt
    server: https://localhost:8443
  name: "minikube"
contexts:
- context:
    cluster: "minikube"
    user: "minikube"
  name: "minikube"
current-context: "minikube"
kind: Config
preferences: {}
users:
- name: "minikube"
  user:
    client-certificate: /var/lib/minikube/certs/apiserver.crt
    client-key: /var/lib/minikube/certs/apiserver.key


In [7]:
CONTEXT=$(kubectl config current-context)
echo $CONTEXT
NAMESPACE=$SOURCENAMESPACE

NEW_CONTEXT=my-context
KUBECONFIG=my-kubeconfig

minikube


In [8]:
SECRET=$(kubectl get serviceaccount ${SERVICEACCOUNT} \
  --context ${CONTEXT} \
  --namespace ${NAMESPACE} \
  -o jsonpath='{.secrets[0].name}')
TOKEN_DATA=$(kubectl get secret ${SECRET} \
  --context ${CONTEXT} \
  --namespace ${NAMESPACE} \
  -o jsonpath='{.data.token}')

TOKEN=$(echo ${TOKEN_DATA} | base64 --decode)



In [9]:
echo $TOKEN_DATA

ZXlKaGJHY2lPaUpTVXpJMU5pSXNJbXRwWkNJNkluazJRalJYYzJKVFRGVlVibE4zWTAweloyUldOWFpsZGtaRmJFUkxOV1IzTUdaaGVVOUZXWEpyZUdzaWZRLmV5SnBjM01pT2lKcmRXSmxjbTVsZEdWekwzTmxjblpwWTJWaFkyTnZkVzUwSWl3aWEzVmlaWEp1WlhSbGN5NXBieTl6WlhKMmFXTmxZV05qYjNWdWRDOXVZVzFsYzNCaFkyVWlPaUp0ZVMxdVlXMWxjM0JoWTJVaUxDSnJkV0psY201bGRHVnpMbWx2TDNObGNuWnBZMlZoWTJOdmRXNTBMM05sWTNKbGRDNXVZVzFsSWpvaWJYa3RjMlZ5ZG1salpXRmpZMjkxYm5RdGRHOXJaVzR0T1RjNGNHY2lMQ0pyZFdKbGNtNWxkR1Z6TG1sdkwzTmxjblpwWTJWaFkyTnZkVzUwTDNObGNuWnBZMlV0WVdOamIzVnVkQzV1WVcxbElqb2liWGt0YzJWeWRtbGpaV0ZqWTI5MWJuUWlMQ0pyZFdKbGNtNWxkR1Z6TG1sdkwzTmxjblpwWTJWaFkyTnZkVzUwTDNObGNuWnBZMlV0WVdOamIzVnVkQzUxYVdRaU9pSXdZamN4Wm1ReVppMHlaVGxoTFRSbU5UTXRPVGd4TVMwellXUXpORFEyWkdKaE1qUWlMQ0p6ZFdJaU9pSnplWE4wWlcwNmMyVnlkbWxqWldGalkyOTFiblE2YlhrdGJtRnRaWE53WVdObE9tMTVMWE5sY25acFkyVmhZMk52ZFc1MEluMC5PZlFkZ2VXWHozNGNUS0ktR25pR25LZ01hS0I4V2tPRW1teWdfOG5TN2RkeWVQOVBZbEhUY3dCVUFRUmVNWllWejRqVlk5NVhnS2RoczhuYVdObDdWZ3R4bmh4WGpZMnhJV1pPYmVTTlJ0OXZjMGZTcDFwNmJjY2tVaTZnVlVmZ0R2amg5LVdPUDRD

## **Create a dedicated kubeconfig**

Starting with a full copy

In [10]:
echo $KUBECONFIG

my-kubeconfig


In [11]:

kubectl config view --raw > ${KUBECONFIG}.full.tmp

Switch working context to correct context

In [12]:
kubectl --kubeconfig ${KUBECONFIG}.full.tmp config use-context ${CONTEXT}

Switched to context "minikube".


Minify and flatten the config

In [13]:
kubectl --kubeconfig ${KUBECONFIG}.full.tmp \
  config view --flatten --minify > ${KUBECONFIG}.tmp

In [14]:
cat ${KUBECONFIG}.tmp

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeE1ERXhNekV4TVRjME9Gb1hEVE14TURFeE1qRXhNVGMwT0Zvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBS0x0CkVtakcrN0FGb1hsNGRUSWFweDJIVVdoM0p1ODBmWDB0ZGNGZVFEQ3JUYzRkTVgrcW83R3Y4d3VYWmhGdWtMSjIKU1FLSXF6S0ZOM0NPbWtPV2FUTGFraWtkWnBHdmZicmlFR2Rxd3R6a1JvYm1UMEZvMndXWkl2ZjE2UFFnZlhGcApjRG5vbXBnVWVhempkK3BQc3RQbWJ6SlBTUy8zMU9iYXNMaVhLMnhrU2l2MUxKQ2hFcHdMYUtPVFc5UXV2R2RkCmt0R2ZmTEI5NFJGSWtXQzhLOHAyZHBCd0hNMlMzcVdOeitmRDhXK3kycllCRDlLTG5xSC81Qkk4T01TTlhtWisKVHZpb3QrSm40MEZzb1NGek9KdWlHeTBvZERFS0NOejAvN1V2Tm14UUZNbFVHN0lWSjgyWGNseDVIM0RaMFFGdwpSY1Z2NFFpSmFONGRSWC9yT3ZVQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NHQVFVRkJ3TUNCZ2dyQmdFRkJRY0RBVEFQQmdOVkhSTUJBZjhFQlRBREFRSC9NQjBHQTFVZERnUVcKQkJScytybmhjN25nVCtjQkV4S0xv

Rename context

In [15]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  rename-context ${CONTEXT} ${NEW_CONTEXT}

Context "minikube" renamed to "my-context".


Set the user credentials for the user created


In [16]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  set-credentials ${CONTEXT}-${NAMESPACE}-token-user \
  --token ${TOKEN}

User "minikube-my-namespace-token-user" set.


Set context to use token user

In [17]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  set-context ${NEW_CONTEXT} --user ${CONTEXT}-${NAMESPACE}-token-user

Context "my-context" modified.


Set context to correct namespace

In [18]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  set-context ${NEW_CONTEXT} --namespace ${NAMESPACE}

Context "my-context" modified.


Flatten/minify kubeconfig


In [19]:
kubectl config --kubeconfig ${KUBECONFIG}.tmp \
  view --flatten --minify > ${KUBECONFIG}

cleanup

In [20]:
rm ${KUBECONFIG}.full.tmp
rm ${KUBECONFIG}.tmp

In [21]:
kubectl --kubeconfig ${KUBECONFIG} get pods

No resources found in my-namespace namespace.


In [22]:
kubectl --kubeconfig ${KUBECONFIG} get pods -A

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "pods" in API group "" at the cluster scope


: 1

In [23]:
cat ${KUBECONFIG}

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeE1ERXhNekV4TVRjME9Gb1hEVE14TURFeE1qRXhNVGMwT0Zvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBS0x0CkVtakcrN0FGb1hsNGRUSWFweDJIVVdoM0p1ODBmWDB0ZGNGZVFEQ3JUYzRkTVgrcW83R3Y4d3VYWmhGdWtMSjIKU1FLSXF6S0ZOM0NPbWtPV2FUTGFraWtkWnBHdmZicmlFR2Rxd3R6a1JvYm1UMEZvMndXWkl2ZjE2UFFnZlhGcApjRG5vbXBnVWVhempkK3BQc3RQbWJ6SlBTUy8zMU9iYXNMaVhLMnhrU2l2MUxKQ2hFcHdMYUtPVFc5UXV2R2RkCmt0R2ZmTEI5NFJGSWtXQzhLOHAyZHBCd0hNMlMzcVdOeitmRDhXK3kycllCRDlLTG5xSC81Qkk4T01TTlhtWisKVHZpb3QrSm40MEZzb1NGek9KdWlHeTBvZERFS0NOejAvN1V2Tm14UUZNbFVHN0lWSjgyWGNseDVIM0RaMFFGdwpSY1Z2NFFpSmFONGRSWC9yT3ZVQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NHQVFVRkJ3TUNCZ2dyQmdFRkJRY0RBVEFQQmdOVkhSTUJBZjhFQlRBREFRSC9NQjBHQTFVZERnUVcKQkJScytybmhjN25nVCtjQkV4S0xv

In [24]:
kubectl --kubeconfig ${KUBECONFIG} get pods 

No resources found in my-namespace namespace.


In [25]:
kubectl --kubeconfig ${KUBECONFIG}  run nginx2 --image=nginx --port=80

pod/nginx2 created


In [27]:
kubectl get pods -A

NAMESPACE      NAME                                          READY   STATUS    RESTARTS   AGE
default        nginx-deployment-6799fc88d8-cthtm             1/1     Running   0          34m
default        nginx-hostnetwork-deployment-db7654df-88wgs   1/1     Running   0          56m
default        nginx-other-deployment-6799fc88d8-27zdq       1/1     Running   0          46m
default        nginx-strict-747fbd9b9b-6bc5m                 1/1     Running   0          40m
kube-system    coredns-74ff55c5b-hh6tc                       1/1     Running   0          3h47m
kube-system    etcd-minikube                                 1/1     Running   0          3h47m
kube-system    kube-controller-manager-minikube              1/1     Running   0          3h47m
kube-system    kube-proxy-djvpf                              1/1     Running   0          3h47m
kube-system    kube-scheduler-minikube                       1/1     Running   0          3h47m
kube-system    storage-provisioner                

In [28]:
kubectl --kubeconfig ${KUBECONFIG} get pods

NAME     READY   STATUS    RESTARTS   AGE
nginx2   1/1     Running   0          36s


In [29]:
kubectl --kubeconfig ${KUBECONFIG} get secrets

NAME                            TYPE                                  DATA   AGE
default-token-z9qrq             kubernetes.io/service-account-token   3      23m
my-serviceaccount-token-978pg   kubernetes.io/service-account-token   3      23m


In [30]:
kubectl --kubeconfig ${KUBECONFIG} --namespace default get deployment

Error from server (Forbidden): deployments.apps is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "deployments" in API group "apps" in the namespace "default"


: 1

In [31]:
kubectl --kubeconfig ${KUBECONFIG}  --namespace default run nginx2 --image=nginx --port=80 

Error from server (Forbidden): pods is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot create resource "pods" in API group "" in the namespace "default"


: 1

In [32]:
cat  ${KUBECONFIG} 

apiVersion: v1
clusters:
- cluster:
    certificate-authority-data: LS0tLS1CRUdJTiBDRVJUSUZJQ0FURS0tLS0tCk1JSURCakNDQWU2Z0F3SUJBZ0lCQVRBTkJna3Foa2lHOXcwQkFRc0ZBREFWTVJNd0VRWURWUVFERXdwdGFXNXAKYTNWaVpVTkJNQjRYRFRJeE1ERXhNekV4TVRjME9Gb1hEVE14TURFeE1qRXhNVGMwT0Zvd0ZURVRNQkVHQTFVRQpBeE1LYldsdWFXdDFZbVZEUVRDQ0FTSXdEUVlKS29aSWh2Y05BUUVCQlFBRGdnRVBBRENDQVFvQ2dnRUJBS0x0CkVtakcrN0FGb1hsNGRUSWFweDJIVVdoM0p1ODBmWDB0ZGNGZVFEQ3JUYzRkTVgrcW83R3Y4d3VYWmhGdWtMSjIKU1FLSXF6S0ZOM0NPbWtPV2FUTGFraWtkWnBHdmZicmlFR2Rxd3R6a1JvYm1UMEZvMndXWkl2ZjE2UFFnZlhGcApjRG5vbXBnVWVhempkK3BQc3RQbWJ6SlBTUy8zMU9iYXNMaVhLMnhrU2l2MUxKQ2hFcHdMYUtPVFc5UXV2R2RkCmt0R2ZmTEI5NFJGSWtXQzhLOHAyZHBCd0hNMlMzcVdOeitmRDhXK3kycllCRDlLTG5xSC81Qkk4T01TTlhtWisKVHZpb3QrSm40MEZzb1NGek9KdWlHeTBvZERFS0NOejAvN1V2Tm14UUZNbFVHN0lWSjgyWGNseDVIM0RaMFFGdwpSY1Z2NFFpSmFONGRSWC9yT3ZVQ0F3RUFBYU5oTUY4d0RnWURWUjBQQVFIL0JBUURBZ0trTUIwR0ExVWRKUVFXCk1CUUdDQ3NHQVFVRkJ3TUNCZ2dyQmdFRkJRY0RBVEFQQmdOVkhSTUJBZjhFQlRBREFRSC9NQjBHQTFVZERnUVcKQkJScytybmhjN25nVCtjQkV4S0xv

In [33]:
kubectl describe serviceaccount -n  my-namespace

Name:                default
Namespace:           my-namespace
Labels:              <none>
Annotations:         <none>
Image pull secrets:  <none>
Mountable secrets:   default-token-z9qrq
Tokens:              default-token-z9qrq
Events:              <none>


Name:                my-serviceaccount
Namespace:           my-namespace
Labels:              <none>
Annotations:         <none>
Image pull secrets:  <none>
Mountable secrets:   my-serviceaccount-token-978pg
Tokens:              my-serviceaccount-token-978pg
Events:              <none>


In [36]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i get pod --namespace default

no


: 1

In [37]:
kubectl  --kubeconfig ${KUBECONFIG} cluster-info


To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
Error from server (Forbidden): services is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "services" in API group "" in the namespace "kube-system"


: 1

In [38]:
kubectl  cluster-info

Kubernetes control plane is running at https://localhost:8443
KubeDNS is running at https://localhost:8443/api/v1/namespaces/kube-system/services/kube-dns:dns/proxy

To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.


In [40]:
kubectl  --kubeconfig ${KUBECONFIG} cluster-info


To further debug and diagnose cluster problems, use 'kubectl cluster-info dump'.
Error from server (Forbidden): services is forbidden: User "system:serviceaccount:my-namespace:my-serviceaccount" cannot list resource "services" in API group "" in the namespace "kube-system"


: 1

# Graphical View

In [40]:
go get github.com/team-soteria/rback

In [41]:
kubectl get sa,roles,clusterroles,clusterrolebindings --all-namespaces -o json |\
rback  -ignore-prefixes none > minikube.dot

In [42]:
apt install graphviz -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  fontconfig fontconfig-config fonts-dejavu-core fonts-liberation libann0
  libcairo2 libcdt5 libcgraph6 libdatrie1 libfontconfig1 libfreetype6
  libfribidi0 libgd3 libgraphite2-3 libgts-0.7-5 libgts-bin libgvc6 libgvpr2
  libharfbuzz0b libice6 libjbig0 libjpeg-turbo8 libjpeg8 liblab-gamut1
  libpango-1.0-0 libpangocairo-1.0-0 libpangoft2-1.0-0 libpathplan4
  libpixman-1-0 libpng16-16 libsm6 libthai-data libthai0 libtiff5 libwebp6
  libx11-6 libx11-data libxau6 libxaw7 libxcb-render0 libxcb-shm0 libxcb1
  libxdmcp6 libxext6 libxmu6 libxpm4 libxrender1 libxt6 x11-common
Suggested packages:
  gsfonts graphviz-doc libgd-tools
The following NEW packages will be installed:
  fontconfig fontconfig-config fonts-dejavu-core fonts-liberation graphviz
  libann0 libcairo2 libcdt5 libcgraph6 libdatrie1 libfontconfig1 libfreetype6
  libfribidi0 libgd3 

In [43]:
dot -Tpng -Grankdir=LR minikube.dot -o minikube.png

![Minikube](minikube.png)

In [47]:
kubectl --kubeconfig ${KUBECONFIG}  auth can-i --list

Resources                                       Non-Resource URLs                     Resource Names   Verbs
*.*                                             []                                    []               [*]
selfsubjectaccessreviews.authorization.k8s.io   []                                    []               [create]
selfsubjectrulesreviews.authorization.k8s.io    []                                    []               [create]
                                                [/.well-known/openid-configuration]   []               [get]
                                                [/api/*]                              []               [get]
                                                [/api]                                []               [get]
                                                [/apis/*]                             []               [get]
                                                [/apis]                               []               [get]
               

In [44]:
kubectl describe clusterrole.rbac  | grep --color=always -E '(create)'


  rolebindings.rbac.authorization.k8s.io          []                 []              [create delete deletecollection get list patch update watch]
  roles.rbac.authorization.k8s.io                 []                 []              [create delete deletecollection get list patch update watch]
  configmaps                                      []                 []              [create delete deletecollection patch update get list watch]
  endpoints                                       []                 []              [create delete deletecollection patch update get list watch]
  persistentvolumeclaims                          []                 []              [create delete deletecollection patch update get list watch]
  pods                                            []                 []              [create delete deletecollection patch update get list watch]
  replicationcontrollers/scale                    []                 []              [create delete deletecollection patch u

In [54]:
kubectl get sa,roles,clusterroles,clusterrolebindings -n my-namespace -o json | \
rback vis --include-namespaces=my-namespace --outformat dot > my-namespace-rbac.dot


In [55]:
dot -Tpng -Grankdir=LR  my-namespace-rbac.dot -o  my-namespace-rbac.png

![ my-namespace-rbac.png](my-namespace-rbac.png)

In [ ]:
kubectl auth can-i --list 

In [56]:
kubectl get sa,roles,clusterroles,clusterrolebindings  -o json | \
rback vis --include-namespaces=my-namespace --outformat dot > defaut-rbac.dot

In [58]:
dot -Tpng -Grankdir=LR  defaut-rbac.dot -o  default-rbac.png

In [1]:
ls *png

openshift-rules.png  rules.png


In [18]:
kubectl  -n my-namespace get sa -o yaml

apiVersion: v1
items:
- apiVersion: v1
  kind: ServiceAccount
  metadata:
    creationTimestamp: "2020-06-05T18:52:04Z"
    name: default
    namespace: my-namespace
    resourceVersion: "460"
    selfLink: /api/v1/namespaces/my-namespace/serviceaccounts/default
    uid: 155da729-f38a-4829-a88a-3f09d6e61b11
  secrets:
  - name: default-token-nchg4
- apiVersion: v1
  kind: ServiceAccount
  metadata:
    creationTimestamp: "2020-06-05T18:52:05Z"
    name: my-serviceaccount
    namespace: my-namespace
    resourceVersion: "467"
    selfLink: /api/v1/namespaces/my-namespace/serviceaccounts/my-serviceaccount
    uid: 305d8346-c5a6-4fff-a6dd-4e30dccc9116
  secrets:
  - name: my-serviceaccount-token-hm2cs
kind: List
metadata:
  resourceVersion: ""
  selfLink: ""


In [12]:
echo $KUBECONFIG

In [20]:
kubectl get rolebinding -A -o yaml

apiVersion: v1
items:
- apiVersion: rbac.authorization.k8s.io/v1
  kind: RoleBinding
  metadata:
    creationTimestamp: "2020-06-05T18:51:43Z"
    name: kubeadm:bootstrap-signer-clusterinfo
    namespace: kube-public
    resourceVersion: "190"
    selfLink: /apis/rbac.authorization.k8s.io/v1/namespaces/kube-public/rolebindings/kubeadm:bootstrap-signer-clusterinfo
    uid: 3fb8a795-5408-4dd8-950c-1ae04f09195a
  roleRef:
    apiGroup: rbac.authorization.k8s.io
    kind: Role
    name: kubeadm:bootstrap-signer-clusterinfo
  subjects:
  - apiGroup: rbac.authorization.k8s.io
    kind: User
    name: system:anonymous
- apiVersion: rbac.authorization.k8s.io/v1
  kind: RoleBinding
  metadata:
    annotations:
      rbac.authorization.kubernetes.io/autoupdate: "true"
    creationTimestamp: "2020-06-05T18:51:42Z"
    labels:
      kubernetes.io/bootstrapping: rbac-defaults
    name: system:controller:bootstrap-signer
    namespace: kube-public
    resourceVersion: "138"
    selfLink: /apis/rbac.

In [65]:
# cleanup
#rm minikube.dot minikube.png my-namespace-rbac.dot my-namespace-rbac.png my-kubeconfig
kubectl config current-context

minikube


In [ ]:
go get -v github.com/thomasfricke/rbac-tool

github.com/thomasfricke/rbac-tool (download)
github.com/alcideio/rbac-tool (download)
get "k8s.io/api/rbac/v1": found meta tag get.metaImport{Prefix:"k8s.io/api", VCS:"git", RepoRoot:"https://github.com/kubernetes/api"} at //k8s.io/api/rbac/v1?go-get=1
get "k8s.io/api/rbac/v1": verifying non-authoritative meta tag
k8s.io/api (download)
github.com/gogo/protobuf (download)
get "k8s.io/apimachinery/pkg/apis/meta/v1": found meta tag get.metaImport{Prefix:"k8s.io/apimachinery", VCS:"git", RepoRoot:"https://github.com/kubernetes/apimachinery"} at //k8s.io/apimachinery/pkg/apis/meta/v1?go-get=1
get "k8s.io/apimachinery/pkg/apis/meta/v1": verifying non-authoritative meta tag
k8s.io/apimachinery (download)
github.com/google/gofuzz (download)
get "gopkg.in/inf.v0": found meta tag get.metaImport{Prefix:"gopkg.in/inf.v0", VCS:"git", RepoRoot:"https://gopkg.in/inf.v0"} at //gopkg.in/inf.v0?go-get=1
gopkg.in/inf.v0 (download)
get "k8s.io/apimachinery/pkg/util/sets": found meta tag get.metaImport{Pre